We built a model in 3.0-theberling-classifier.ipynb but we didn't have the right submissions. So now we're going to reverse engineer from their sample submission file to get the year, and team IDs that we need. Let's start by reading in their sample submission data file.

In [1]:
import pandas as pd

In [2]:
base_path = "/kaggle/input/march-machine-learning-mania-2023/"
df_submission_sample = pd.read_csv(base_path + 'SampleSubmission2023.csv')

In [3]:
df_submission_sample.head()

,ID,Pred
0,2023_1101_1102,0.5
1,2023_1101_1103,0.5
2,2023_1101_1104,0.5
3,2023_1101_1105,0.5
4,2023_1101_1106,0.5


In [4]:
new = df_submission_sample["ID"].str.split("_", expand = True)
new

,0,1,2
0,2023,1101,1102
1,2023,1101,1103
2,2023,1101,1104
3,2023,1101,1105
4,2023,1101,1106
...,...,...,...
130678,2023,3474,3476
130679,2023,3474,3477
130680,2023,3475,3476
130681,2023,3475,3477


In [5]:
combinations = pd.DataFrame()
combinations['Season'] = new[0]
combinations['LowID'] = new[1]
combinations['HighID'] = new[2]
combinations['Season'] = pd.to_numeric(combinations['Season'])
combinations['LowID'] = pd.to_numeric(combinations['LowID'])
combinations['HighID'] = pd.to_numeric(combinations['HighID'])
combinations.head()

,Season,LowID,HighID
0,2023,1101,1102
1,2023,1101,1103
2,2023,1101,1104
3,2023,1101,1105
4,2023,1101,1106


Now let's load up the season averages data.

In [6]:
df = pd.read_csv('../input/2-compute-averages/data_averages.csv')

Let's merge the low ID team with it's data by season. To do that we'll temporarily rename LowID to TeamID and merge on Season and TeamID.

In [7]:
LowID = pd.merge(combinations.rename({'LowID': 'TeamID'}, axis=1), df, how="left", on=['Season','TeamID']).dropna().reset_index(drop=True)
LowID.head()

,Season,TeamID,HighID,Score,FGP,FGP3,FTP,OR,DR,Ast,TO,Stl,Blk,PF,Rank,Seed


In [8]:
LowID.columns = ['Season','LowID','HighID','LowScore','LowFGP','LowFGP3','LowFTP','LowOR','LowDR','LowAst','LowTO','LowStl','LowBlk','LowPF','LowRank','LowSeed']
LowID.head()

,Season,LowID,HighID,LowScore,LowFGP,LowFGP3,LowFTP,LowOR,LowDR,LowAst,LowTO,LowStl,LowBlk,LowPF,LowRank,LowSeed


In [9]:
HighID = pd.merge(combinations.rename({'HighID': 'TeamID'}, axis=1), df, how="left", on=['Season','TeamID']).dropna().reset_index(drop=True)
HighID.head()

,Season,LowID,TeamID,Score,FGP,FGP3,FTP,OR,DR,Ast,TO,Stl,Blk,PF,Rank,Seed


In [10]:
HighID.columns = ['Season','LowID','HighID','HighScore','HighFGP','HighFGP3','HighFTP','HighOR','HighDR','HighAst','HighTO','HighStl','HighBlk','HighPF','HighRank','HighSeed']
HighID.head()

,Season,LowID,HighID,HighScore,HighFGP,HighFGP3,HighFTP,HighOR,HighDR,HighAst,HighTO,HighStl,HighBlk,HighPF,HighRank,HighSeed


And finally we can merge the HighID and LowID DataFrame on Season, LowID, HighID, and Win since those should be the same in each DataFrame.

In [11]:
model_dataset = pd.merge(LowID, HighID, how='outer', on=['Season','LowID','HighID'])
model_dataset.head()

,LowScore,LowFGP,LowFGP3,LowFTP,LowOR,LowDR,LowAst,LowTO,LowStl,LowBlk,...,HighFTP,HighOR,HighDR,HighAst,HighTO,HighStl,HighBlk,HighPF,HighRank,HighSeed


In [12]:
model_dataset.shape

(0, 29)

In [13]:
model_dataset.to_csv('/kaggle/working/model_dataset2.csv',index=False)